In [1]:
import torch

print(torch.cuda.get_device_name())

Tesla P100-PCIE-16GB


In [2]:
!unzip -q '/content/drive/MyDrive/test_data.zip' -d '/content'
!unzip -q '/content/drive/MyDrive/train_data.zip' -d '/content'

In [20]:
from torchvision.transforms.transforms import RandomAutocontrast
# https://www.youtube.com/watch?v=Jy4wM2X21u0&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=3

# Imports
import torch
import torch.nn as nn 
import torch.optim as optim # optimization algos (SGD, Adam, etc.)
import torch.nn.functional as F # activation function, etc.
from torch.utils.data import Dataset, DataLoader # easier data management and creates minibatches
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)

# training set data augmentation
train_transforms = transforms.Compose([
    transforms.ColorJitter(brightness=0.5),
    transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomEqualize(p=0.5),
    transforms.RandomAutocontrast(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.3826, 0.3692, 0.3057], std=[0.3249, 0.3166, 0.3029])
])

# test set transformations
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.3820, 0.3704, 0.3064], std=[0.3255, 0.3179, 0.3040])
])

# Load Data
TRAIN_DATA_PATH = '/content/train_data/train_img'
TEST_DATA_PATH = '/content/test_data/test_img'

batch_size = 64

train_data = datasets.ImageFolder(root = TRAIN_DATA_PATH, transform=train_transforms)
train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
test_data = datasets.ImageFolder(root = TEST_DATA_PATH, transform=test_transforms)
test_data_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4)

# CNN model
class CNN(nn.Module):
    def __init__(self, in_channels = 3, hidden_size1 = 2048, num_classes = 10):
        super(CNN, self).__init__()
        # same convolution
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        
        self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
        
        self.fc1 = nn.Linear(256*16*16, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes)

        self.dropout = nn.Dropout(0.5)

        self.bn1 = nn.BatchNorm1d(hidden_size1)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        x = F.relu(self.conv5(x))
        #x = self.pool(x)
        x = x.reshape(x.shape[0], -1)

        x = self.dropout(x)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = F.softmax(self.fc2(x), dim=1)
        
        return x
    
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_uniform_(m.weight)

                if m.bias is not None:
                    nn.init.constant_(m_bias, 0)

model = CNN()
exit()

# Hyperparameters
learning_rate = 0.0001
num_epochs = 10

x_epoch, y_epoch = [0], [0]
x_step, y_step = [0], [0] 

# Initializer network
model = CNN().to(device)

# Loss and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train network
total_steps = 1
model.train()
for epoch in range(num_epochs):
    epoch_num_correct = 0
    epoch_num_samples = 0

    for x, y in train_data_loader:
        # Get data to cuda if possible
        x = x.to(device=device)
        y = y.to(device=device)
        
        # Get to correct shape
        # x = x.reshape(x.shape[0], -1)
        
        #forward prop
        scores = model(x)
        loss = loss_func(scores, y)
        
        #backprop
        optimizer.zero_grad()
        loss.backward()
        
        # GD or adam step
        optimizer.step()
    
        num_correct = 0
        num_samples = 0

        if total_steps%batch_size == 0:
            with torch.no_grad():
                for x, y in train_data_loader:
                    x = x.to(device=device)
                    y = y.to(device=device)
                    # x = x.reshape(x.shape[0], -1)
                    
                    scores = model(x)
                    _, predictions = scores.max(1)

                    num_correct += (predictions == y).sum()
                    num_samples += predictions.size(0)
                    
                    epoch_num_correct += (predictions == y).sum()                 
                    epoch_num_samples += predictions.size(0)
            
            cur_accuracy = (num_correct)/(num_samples)*100
            batch_num = int(total_steps/batch_size)

            x_step.append(batch_num)
            y_step.append(cur_accuracy.cpu().numpy())

            print(f'Accuracy after Batch #{batch_num}:\t{cur_accuracy:.2f}%')

        total_steps += 1

    epoch_accuracy = (epoch_num_correct)/(epoch_num_samples)*100

    x_epoch.append(epoch+1)
    y_epoch.append(epoch_accuracy.cpu().numpy())

    print(f'Epoch {epoch+1} | Accuracy ----------------------------> {epoch_accuracy:.2f}%')

plt.plot(x_step, y_step)
plt.ylabel('Accuracy (%)')
plt.xlabel('Batch Number')
plt.title('Vanilla CNN for Animals10')
plt.show()

print('\n\n')

plt.plot(x_epoch, y_epoch)
plt.title('Vanilla CNN for Animals10')
plt.ylabel('Accuracy (%)')
plt.xlabel('Epochs')
plt.show()

#########################################

# Test network
test_num_correct = 0
test_num_samples = 0
model.eval()

with torch.no_grad():
    for x, y in test_data_loader:
        x = x.to(device=device)
        y = y.to(device=device)

        scores = model(x)
        _, predictions = scores.max(1)
        test_num_correct += (predictions == y).sum()
        test_num_samples += predictions.size(0)
    
    print('*' * 30)
    print(f'Test Accuracy ----------------------------> {(test_num_correct)/(test_num_samples)*100:.2f}%')



Accuracy after Batch #1:	31.56%


KeyboardInterrupt: ignored